In [ ]:
# =============================================================================
# SCRIPT: build_ndvi_features.py
# PURPOSE: Extracts and computes local, grid, and delta NDVI + land-cover
#          fractions (water/urban/suburban/forest) for each station.
# =============================================================================

import os
import glob
import pandas as pd
import numpy as np
import rasterio
from rasterio.windows import from_bounds
from pyproj import Transformer
from tqdm import tqdm
import re
import gc
import xarray as xr

In [ ]:
# =============================================================================
# PART 1: CONFIGURATION
# =============================================================================

DATA_FOLDER = r"C:\Users\drugm\Documents\GENHACK\GENHACK\data"

NDVI_FOLDER = os.path.join(DATA_FOLDER, "sentinel2_ndvi")
ECA_FOLDER = os.path.join(DATA_FOLDER, "ECA_blend_tx")
STATIONS_FILE = os.path.join(ECA_FOLDER, "stations.txt")

TARGET_COUNTRIES = ['ES', 'IT', 'FR', 'PT']

WATER_MAX     = 0.10  # NDVI < 0.10
URBAN_MAX     = 0.33  # 0.10 <= NDVI < 0.33
SUBURBAN_MAX  = 0.60  # 0.33 <= NDVI < 0.60
# NDVI >= 0.60 -> forest

In [ ]:
# =============================================================================
# PART 2: LOAD STATION METADATA
# =============================================================================

def dms_to_decimal(dms_str):
    """Converts a Degrees:Minutes:Seconds string to decimal degrees."""
    if not isinstance(dms_str, str):
        return None
    original_dms_str = dms_str
    dms_str = dms_str.strip().lstrip('+-')
    parts = dms_str.split(':')
    if len(parts) < 3:
        return None
    val = float(parts[0]) + float(parts[1]) / 60 + float(parts[2]) / 3600
    return -val if '-' in original_dms_str else val

print("--- Loading Station Metadata ---\n")

if not os.path.exists(STATIONS_FILE):
    raise FileNotFoundError(f"Error: Station file not found at {STATIONS_FILE}")
else:
    df_meta = pd.read_csv(STATIONS_FILE, skiprows=17, skipinitialspace=True)
    df_meta.columns = [c.strip() for c in df_meta.columns]
    df_meta['CN'] = df_meta['CN'].str.strip()

    df_meta = df_meta[df_meta['CN'].isin(TARGET_COUNTRIES)].copy()

    df_meta['lat_dec'] = df_meta['LAT'].apply(dms_to_decimal)
    df_meta['lon_dec'] = df_meta['LON'].apply(dms_to_decimal)
    df_meta.dropna(subset=['lat_dec', 'lon_dec'], inplace=True)

    station_locs = df_meta.set_index('STAID')[['lat_dec', 'lon_dec', 'HGHT']].to_dict('index')

    print(f"Found and processed {len(station_locs)} stations across {TARGET_COUNTRIES}.")

In [ ]:
# =============================================================================
# PART 3: SOPHISTICATED NDVI EXTRACTION (Grid vs. Point)
# =============================================================================
ERA5_GRID_SIZE =0.1
HALF_GRID = ERA5_GRID_SIZE / 2.0

station_mapping = []
for staid, loc in station_locs.items():
    era5_lat_center = round(loc['lat_dec'] / ERA5_GRID_SIZE) * ERA5_GRID_SIZE
    era5_lon_center = round(loc['lon_dec'] / ERA5_GRID_SIZE) * ERA5_GRID_SIZE

    station_mapping.append({
        'STAID': staid,
        'st_lat': loc['lat_dec'],
        'st_lon': loc['lon_dec'],
        'box_min_lat': era5_lat_center - HALF_GRID,
        'box_max_lat': era5_lat_center + HALF_GRID,
        'box_min_lon': era5_lon_center - HALF_GRID,
        'box_max_lon': era5_lon_center + HALF_GRID
    })
df_geom = pd.DataFrame(station_mapping)

ndvi_stats = {
    staid: {
        'local': [],
        'grid': [],
        'water_frac': [],
        'urban_frac': [],
        'suburban_frac': [],
        'forest_frac': []
    }
    for staid in station_locs.keys()
}

ndvi_files=glob.glob(os.path.join(NDVI_FOLDER, "*.tif"))

for tiff_path in tqdm(ndvi_files, desc="Processing Satellite Maps"):
    match = re.search(r'(\d{4}-\d{2}-\d{2})', os.path.basename(tiff_path))
    if not match:
        continue
    timestamp = pd.to_datetime(match.group(1))

    if timestamp.month not in [6, 7, 8, 9]:
        continue

    try:
        with rasterio.open(tiff_path) as src:
            transformer = Transformer.from_crs("EPSG:4326", src.crs, always_xy=True)

            for _, row in df_geom.iterrows():
                staid = row['STAID']

                try:
                    x_point, y_point = transformer.transform(row['st_lon'], row['st_lat'])
                    gen = src.sample([(x_point, y_point)])
                    raw_val = next(gen)[0]
                    if raw_val != 255:
                        ndvi_local = (raw_val / 254.0 * 2.0) - 1.0
                        if -1 <= ndvi_local <= 1:
                            ndvi_stats[staid]['local'].append(ndvi_local)
                except Exception:
                    continue

                try:
                    lons = [row['box_min_lon'], row['box_max_lon']]
                    lats = [row['box_min_lat'], row['box_max_lat']]
                    xs, ys = transformer.transform(lons, lats)
                    left, bottom, right, top = min(xs), min(ys), max(xs), max(ys)

                    window = from_bounds(left, bottom, right, top, transform=src.transform)
                    data = src.read(1, window=window, boundless=True, fill_value=255)

                    valid_data = data[data != 255]
                    if valid_data.size > 0:
                        valid_float = (valid_data / 254.0 * 2.0) - 1.0
                        valid_float = valid_float[(valid_float >= -1.0) & (valid_float <= 1.0)]
                        if valid_float.size == 0:
                            continue

                        grid_mean = np.mean(valid_float)
                        if -1 <= grid_mean <= 1:
                            ndvi_stats[staid]['grid'].append(grid_mean)


                        water_mask = (valid_float < WATER_MAX)
                        water_frac = np.mean(water_mask)
                        ndvi_stats[staid]['water_frac'].append(water_frac)

                        land_pixels = valid_float[~water_mask]
                        if land_pixels.size == 0:
                            ndvi_stats[staid]['urban_frac'].append(0.0)
                            ndvi_stats[staid]['suburban_frac'].append(0.0)
                            ndvi_stats[staid]['forest_frac'].append(0.0)
                        else:
                            urban_mask_land    = (land_pixels < URBAN_MAX) 
                            suburban_mask_land = (land_pixels >= URBAN_MAX) & (land_pixels < SUBURBAN_MAX)
                            forest_mask_land   = (land_pixels >= SUBURBAN_MAX)

                            ndvi_stats[staid]['urban_frac'].append(np.mean(urban_mask_land))
                            ndvi_stats[staid]['suburban_frac'].append(np.mean(suburban_mask_land))
                            ndvi_stats[staid]['forest_frac'].append(np.mean(forest_mask_land))

                except Exception:
                    continue

    except Exception as e:
        print(f"Skipping file {os.path.basename(tiff_path)} due to error: {e}")

In [ ]:
# =============================================================================
# PART 4: AGGREGATE RESULTS AND SAVE
# =============================================================================

final_ndvi_data = []
for staid, data in ndvi_stats.items():
    mean_local = np.mean(data['local']) if data['local'] else np.nan
    mean_grid  = np.mean(data['grid'])  if data['grid']  else np.nan

    if np.isnan(mean_local) or np.isnan(mean_grid):
        continue

    water_mean = np.mean(data['water_frac'])    if data['water_frac']    else np.nan
    urban_mean  = np.mean(data['urban_frac'])    if data['urban_frac']    else np.nan
    suburban_mean = np.mean(data['suburban_frac']) if data['suburban_frac'] else np.nan
    forest_mean= np.mean(data['forest_frac'])   if data['forest_frac']   else np.nan

    final_ndvi_data.append({
        'STAID': staid,
        'ndvi_local':   mean_local,
        'ndvi_global':  mean_grid,
        'delta_ndvi':   mean_local - mean_grid,
        'perc_water':   water_mean,
        'perc_urban':   urban_mean,
        'perc_suburban': suburban_mean,
        'perc_forest':  forest_mean
    })

df_ndvi_static = pd.DataFrame(final_ndvi_data)

output_filename = "ndvi_static_features.csv"
df_ndvi_static.to_csv(output_filename, index=False)

print(f"\n SUCCESS! Calculated Grid vs Local NDVI for {len(df_ndvi_static)} stations.")
print(f"Data saved to '{output_filename}'.")
print("\n--- Data Preview ---")
print(df_ndvi_static.head())


--- Loading Station Metadata ---

✅ Found and processed 2210 stations across ['ES', 'IT', 'FR', 'PT'].

--- 🌿 Starting Grid vs. Point NDVI Extraction ---


Processing Satellite Maps: 100%|██████████| 16/16 [00:41<00:00,  2.62s/it]


✅ SUCCESS! Calculated Grid vs Local NDVI for 2104 stations.
Data saved to 'ndvi_static_features.csv'.

--- Data Preview ---
   STAID  ndvi_local  ndvi_global  delta_ndvi  perc_water  perc_urban  \
0     31    0.281496     0.260962    0.020534    0.169654    0.554595   
1     32    0.518701     0.466386    0.052315    0.005724    0.224091   
2     33    0.310039     0.356091   -0.046052    0.047130    0.383959   
3     34    0.392717     0.552955   -0.160238    0.014184    0.119336   
4     36    0.303150     0.319731   -0.016582    0.045924    0.549538   

   perc_suburban  perc_forest  
0       0.412429     0.032976  
1       0.531286     0.244623  
2       0.552516     0.063524  
3       0.437106     0.443558  
4       0.405299     0.045163  


In [ ]:
# =============================================================================
# CELL 1: DATA PRE-LOADING (WITH CACHING)
# =============================================================================


DATA_FOLDER = r"C:\Users\drugm\Documents\GENHACK\GENHACK\data"
ERA5_FOLDER = os.path.join(DATA_FOLDER, "derived-era5-land-daily-statistics")
ECA_FOLDER = os.path.join(DATA_FOLDER, "ECA_blend_tx")
START_DATE = "2020-01-01"
END_DATE = "2025-12-31"

ERA5_CACHE_PATH = os.path.join(DATA_FOLDER, "era5_preloaded_cache.parquet")

if os.path.exists(ERA5_CACHE_PATH):
    print(f"- Found cache file. Loading pre-processed ERA5 data from:\n{ERA5_CACHE_PATH} ---")
    
    cached_df = pd.read_parquet(ERA5_CACHE_PATH)
    era5_dfs = {}
    
    for staid in tqdm(cached_df['STAID'].unique(), desc="Reconstructing ERA5 dictionary from cache"):
        station_df = (
            cached_df[cached_df['STAID'] == staid]
            .drop(columns=['STAID'])
            .set_index('time')
        )
        era5_dfs[staid] = station_df
    
    print(f"\nSuccessfully loaded cached ERA5 data for {len(era5_dfs)} stations.")

else:
    print("--No cache file found. Starting 20-minute pre-load from source .nc files... ---")
    
    era5_files=glob.glob(os.path.join(ERA5_FOLDER, "*.nc"))
    ds = xr.open_mfdataset(era5_files, combine='by_coords', parallel=True)
    if 'valid_time' in ds.coords:
        ds = ds.rename({'valid_time': 'time'})
    rename_map = {
        k: v for k, v in {
            'mx2t': 'era5_temp_max',
            't2m' : 'era5_temp_max',
            'tp'  : 'era5_precip',
            'u10' : 'era5_u10',
            'v10' : 'era5_v10'
        }.items() if k in ds
    }
    ds = ds.rename(rename_map)

    era5_dfs = {}
    for staid in tqdm(station_locs.keys(), desc="Extracting ERA5 Time Series"):
        try:
            loc = station_locs[staid]
            ds_point = ds.sel(
                latitude=loc['lat_dec'],
                longitude=loc['lon_dec'],
                method='nearest'
            ).load()
            df_point = ds_point.to_dataframe()
            df_point.reset_index(inplace=True)
            df_point['time'] = pd.to_datetime(df_point['time']).dt.normalize()
            df_point = df_point.drop_duplicates(subset=['time']).set_index('time')

            if 'era5_temp_max' in df_point.columns:
                df_point['era5_temp_max'] -= 273.15 
            if 'era5_precip' in df_point.columns:
                df_point['era5_precip'] *= 1000 

            era5_dfs[staid] = df_point
        except Exception as e:
            print(f"Could not process ERA5 for station {staid}: {e}")
    
    print(f"\n Successfully pre-loaded ERA5 data for {len(era5_dfs)} stations.")
    
    print("\n-Saving processed ERA5 data to cache for future runs... ---")
    if era5_dfs:
        temp_list =[]
        for staid, df in era5_dfs.items():
            df_copy = df.reset_index().copy() 
            df_copy['STAID'] = staid
            temp_list.append(df_copy)
        
        combined_df_to_cache = pd.concat(temp_list)
        combined_df_to_cache.to_parquet(ERA5_CACHE_PATH)
        print(f"ERA5 data cached successfully to:\n{ERA5_CACHE_PATH}")
    else:
        print("No ERA5 data was loaded, so the cache file was not created.")

print("\n--- Loading NDVI static features ---")
ndvi_static_file = "ndvi_static_features.csv"
df_ndvi_static = pd.read_csv(ndvi_static_file).set_index('STAID')
print("NDVI data loaded.")


--- 🧠 Found cache file. Loading pre-processed ERA5 data from:
C:\Users\drugm\Documents\GENHACK\GENHACK\data\era5_preloaded_cache.parquet ---


Reconstructing ERA5 dictionary from cache:   0%|          | 0/2210 [00:00<?, ?it/s]


✅ Successfully loaded cached ERA5 data for 2210 stations.

--- 🍂 Loading NDVI static features ---
✅ NDVI data loaded.


In [ ]:
# =============================================================================
# CELL 2: MERGE ALL DATA SOURCES
# =============================================================================
print("\n--- STEP 2: Starting final dataset construction ---")
training_list = []
TARGET_STATIONS = list(station_locs.keys())

for staid in tqdm(TARGET_STATIONS, desc="Merging All Datasets"):
    try:
        if staid not in era5_dfs:
            continue
        df_e5 = era5_dfs[staid]

        fpath = os.path.join(ECA_FOLDER, f"TX_STAID{int(staid):06d}.txt")
        if not os.path.exists(fpath):
            continue

        df_st = pd.read_csv(fpath, skiprows=20, skipinitialspace=True)
        df_st['time'] = pd.to_datetime(df_st['DATE'], format='%Y%m%d', errors='coerce')
        df_st.dropna(subset=['time'], inplace=True)
        df_st = df_st[df_st['Q_TX'] == 0]
        df_st['TX'] = df_st['TX'] * 0.1  
        df_st.set_index('time', inplace=True)
        df_st = df_st.loc[START_DATE:END_DATE]
        if df_st.empty:
            continue

        merged = pd.merge(df_st[['TX']], df_e5, left_index=True, right_index=True, how='inner')
        if merged.empty:
            continue

        merged.reset_index(inplace=True) 

        merged['STAID'] = staid
        merged = pd.merge(merged, df_ndvi_static, on='STAID', how='left')
        training_list.append(merged)

    except Exception as e:
        print(f"Skipping station {staid} due to an unexpected error: {e}")
        continue

print(f"\n-- Merged data for {len(training_list)} stations. Ready for final processing. ---")



--- 🚀 STEP 2: Starting final dataset construction ---


Merging All Datasets:   0%|          | 0/2210 [00:00<?, ?it/s]


--- 🔧 Merged data for 2110 stations. Ready for final processing. ---


In [ ]:
# =============================================================================
# CELL 3: FINAL CALCULATIONS & SAVE (UPDATED FEATURES)
# =============================================================================
if training_list:
    print("--- Finalizing dataset... ---")

    final_df = pd.concat(training_list, ignore_index=True)

    final_df['time']=pd.to_datetime(final_df['time']).dt.normalize()
    final_df.sort_values(by=['STAID', 'time'], inplace=True)

    final_df['delta_temp'] = final_df['TX'] - final_df['era5_temp_max']
    final_df['doy'] = final_df['time'].dt.dayofyear
    final_df['sin_day'] = np.sin(2 * np.pi * final_df['doy'] / 366.0)
    final_df['cos_day'] = np.cos(2 * np.pi * final_df['doy'] / 366.0)

    final_df['wind_speed'] = np.sqrt(final_df['era5_u10']**2 + final_df['era5_v10']**2)

    final_df['rain_7day_avg'] = final_df.groupby('STAID')['era5_precip'].transform(
        lambda x: x.ewm(span=7).mean()
    )
    final_df['elevation'] = final_df['STAID'].map({k: v['HGHT'] for k, v in station_locs.items()})

    final_columns = [
        'time', 'STAID', 'latitude', 'longitude', 'elevation',
        'TX', 'era5_temp_max', 'delta_temp',
        'ndvi_local', 'ndvi_global', 'delta_ndvi',
        'perc_water', 'perc_urban', 'perc_suburban', 'perc_forest',
        'era5_u10', 'era5_v10', 'wind_speed',
        'era5_precip', 'rain_7day_avg',
        'sin_day', 'cos_day'
    ]
    final_columns_exist = [col for col in final_columns if col in final_df.columns]
    final_df = final_df[final_columns_exist]
    final_df.dropna(subset=['delta_temp', 'delta_ndvi'], inplace=True)

    # Save
    output_filename = "euro_final_dataset_v2.csv"
    final_df.to_csv(output_filename, index=False)

    print(f"\n SUCCESS! Your final dataset is ready! ")
    print(f"Saved {len(final_df)} rows to '{output_filename}'.")
    print("\n--- Final Dataset Preview ---")
    print(final_df.head())
else:
    print("\nCRITICAL ERROR: No data was merged. Check your file paths and station lists.")

for name in ['era5_dfs', 'training_list', 'final_df', 'ds']:
    if name in globals():
        del globals()[name]
gc.collect()


--- Finalizing dataset... ---

✅✅✅ SUCCESS! Your final dataset is ready! ✅✅✅
Saved 3034327 rows to 'euro_final_dataset_v2.csv'.

--- Final Dataset Preview ---
        time  STAID  latitude  longitude  elevation    TX  era5_temp_max  \
0 2020-01-01     32      47.1        2.4        161   7.1       8.108673   
1 2020-01-02     32      47.1        2.4        161   9.2       9.584625   
2 2020-01-03     32      47.1        2.4        161  10.9      10.435638   
3 2020-01-04     32      47.1        2.4        161   8.5       8.464508   
4 2020-01-05     32      47.1        2.4        161   9.0       7.048462   

   delta_temp  ndvi_local  ndvi_global  ...  perc_urban  perc_suburban  \
0   -1.008673    0.518701     0.466386  ...    0.224091       0.531286   
1   -0.384625    0.518701     0.466386  ...    0.224091       0.531286   
2    0.464362    0.518701     0.466386  ...    0.224091       0.531286   
3    0.035492    0.518701     0.466386  ...    0.224091       0.531286   
4    1.951538 

29